# Earth System Simulation - Google Colab Version

This notebook shows how to run the Earth system simulation in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/earth_system_sim/blob/main/examples/colab_simulation.ipynb)

## 1. Setup Environment

First, let's clone the repository and install required packages.

In [ ]:
# Clone repository
!git clone https://github.com/yourusername/earth_system_sim.git
%cd earth_system_sim

# Install package in development mode
!pip install -e .

## 2. Import Dependencies

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
from tqdm.notebook import tqdm
import sys

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from scripts.run_simulation import EarthSystemSimulation
from scripts.visualize_results import create_visualizations

## 3. Mount Google Drive (Optional)

If you want to save results persistently, mount your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create output directory in Drive
import os
output_dir = '/content/drive/MyDrive/earth_system_results'
os.makedirs(output_dir, exist_ok=True)

## 4. Check GPU Availability

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

## 5. Load and Modify Configuration

In [ ]:
# Load config
with open('config/model_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Adjust for Colab resources
config['grid_height'] = 16  # Reduced size
config['grid_width'] = 16   # Reduced size

# Save modified config
with open('config/colab_config.yaml', 'w') as f:
    yaml.dump(config, f)

## 6. Run Simulation

In [ ]:
# Create and run simulation
sim = EarthSystemSimulation('config/colab_config.yaml', device)

print("Running simulation...")
trajectory = sim.run_simulation(
    num_steps=50,
    save_frequency=5
)

print("Simulation complete!")

## 7. Create Visualizations

In [ ]:
# Load visualization config
with open('config/visualization_config.yaml', 'r') as f:
    viz_config = yaml.safe_load(f)

# Create visualizations
print("Creating visualizations...")
create_visualizations(
    trajectory,
    viz_config,
    output_dir,
    make_animations=True
)

print(f"Results saved to: {output_dir}")

## 8. Display Results

In [ ]:
# Plot time series of key variables
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# Physical system - mean temperature
temp_mean = np.mean(trajectory['physical_states'][..., 0], axis=(1, 2))
axes[0].plot(temp_mean, 'r-')
axes[0].set_title('Mean Temperature')
axes[0].grid(True)

# Biosphere - mean vegetation cover
veg_mean = np.mean(trajectory['biosphere_states'][..., 0], axis=0)
axes[1].plot(veg_mean, 'g-')
axes[1].set_title('Mean Vegetation Cover')
axes[1].grid(True)

# Geosphere - mean elevation
elev_mean = np.mean(trajectory['geosphere_states'][..., 0], axis=(1, 2))
axes[2].plot(elev_mean, 'b-')
axes[2].set_title('Mean Elevation')
axes[2].grid(True)

plt.tight_layout()
plt.show()

## 9. Save Results

In [ ]:
# Save simulation results
save_path = os.path.join(output_dir, 'simulation_results.npz')
np.savez(
    save_path,
    physical_states=trajectory['physical'],
    biosphere_states=trajectory['biosphere'],
    geosphere_states=trajectory['geosphere'],
    times=trajectory['times']
)

print(f"Results saved to: {save_path}")